In [1]:
import os, sys
from pathlib import Path
import cortex
import nibabel as nib
import numpy as np
import matplotlib.colors as colors
import matplotlib.pyplot as pl
import time
import platform
import subprocess

In [2]:
if platform.node()=='theCryingMachine.local':
    data_dir = Path('/Users/ningkai/Library/CloudStorage/OneDrive-Personal/Workbench/suppr_pRF/data/spinoza/sub-002/derivatives/')
elif platform.node()=='Smoldering-Corpse-Bar.local':
    data_dir = Path('/Users/dionysus/Library/CloudStorage/OneDrive-Personal/Workbench/suppr_pRF/data/spinoza/sub-002/derivatives/')
elif platform.node()=='node230':
    raise ValueError('hmmm I dont think it works on a server. Your are working on {}'.format(platform.node()))
else:
    raise ValueError('No idea which computer this is. The computer name is {}'.format(platform.node()))

In [3]:
subj_nr = 2
sub = 'sub-{0:03d}'.format(subj_nr)
cx_sub = "EO_" + sub

fs_dir = data_dir / 'freesurfer'
fs_subj_dir = Path(fs_dir, sub)

GLMsingle_dir = data_dir / 'GLMsingle'
example_func_fn = data_dir / 'func' / 'sub-002_ses-01_task-ping_run-6_space-T1w_boldref.nii.gz'

bref = nib.load(example_func_fn)

ctx_dir = data_dir / 'cortex' / 'outputs'

In [4]:
print('cortex.database.default_filestore: {}'.format(cortex.database.default_filestore))
print('cortex.options.usercfg: {}'.format(cortex.options.usercfg))

cortex.database.default_filestore: /Users/ningkai/Documents/pycortex/db
cortex.options.usercfg: /Users/ningkai/Library/Application Support/pycortex/options.cfg


In [5]:
cortex.freesurfer.import_subj(fs_subject=sub,
                                cx_subject=cx_sub,
                                freesurfer_subject_dir=fs_dir)

b'created by nwa201 on Tue Dec 12 21:21:24 2023\n'
b'created by nwa201 on Wed Dec 13 01:55:48 2023\n'
b'created by nwa201 on Tue Dec 12 23:48:23 2023\n'
b'created by nwa201 on Wed Dec 13 02:00:33 2023\n'
b'created by nwa201 on Tue Dec 12 21:21:24 2023\n'
b'created by nwa201 on Tue Dec 12 23:48:23 2023\n'
b'created by nwa201 on Wed Dec 13 01:55:48 2023\n'
b'created by nwa201 on Wed Dec 13 02:00:33 2023\n'
b'created by nwa201 on Tue Dec 12 21:23:11 2023\n'
b'created by nwa201 on Tue Dec 12 23:49:12 2023\n'


In [6]:
transform = cortex.xfm.Transform(np.identity(4), bref)
transform.save(cx_sub, 'fmriprep', 'magnet')

In [7]:
T1 = Path(fs_dir, sub, "mri", "T1.nii.gz")
T2 = Path(fs_dir, sub, "mri", "T2.nii.gz")
T12 = Path(fs_dir, sub, "mri", "T12.nii.gz")
os.system("mri_convert %s %s" % (Path(str(T1).replace('.nii.gz', '.mgz')), T1))
os.system("mri_convert %s %s" % (Path(str(T2).replace('.nii.gz', '.mgz')), T2))
os.system("fslmaths %s -div %s %s" % (T1, T2, T12))
aref = nib.load(T1)
transform = cortex.xfm.Transform(np.identity(4), aref)
transform.save(cx_sub, 'anat', 'magnet')

mri_convert /Users/ningkai/Library/CloudStorage/OneDrive-Personal/Workbench/suppr_pRF/data/spinoza/sub-002/derivatives/freesurfer/sub-002/mri/T1.mgz /Users/ningkai/Library/CloudStorage/OneDrive-Personal/Workbench/suppr_pRF/data/spinoza/sub-002/derivatives/freesurfer/sub-002/mri/T1.nii.gz 
reading from /Users/ningkai/Library/CloudStorage/OneDrive-Personal/Workbench/suppr_pRF/data/spinoza/sub-002/derivatives/freesurfer/sub-002/mri/T1.mgz...
TR=0.00, TE=0.00, TI=0.00, flip angle=0.00
i_ras = (-1, 0, 0)
j_ras = (0, 2.14097e-08, -1)
k_ras = (0, 1, 2.14097e-08)
writing to /Users/ningkai/Library/CloudStorage/OneDrive-Personal/Workbench/suppr_pRF/data/spinoza/sub-002/derivatives/freesurfer/sub-002/mri/T1.nii.gz...
mri_convert /Users/ningkai/Library/CloudStorage/OneDrive-Personal/Workbench/suppr_pRF/data/spinoza/sub-002/derivatives/freesurfer/sub-002/mri/T2.mgz /Users/ningkai/Library/CloudStorage/OneDrive-Personal/Workbench/suppr_pRF/data/spinoza/sub-002/derivatives/freesurfer/sub-002/mri/T2.ni

In [8]:
T12_anat_vcx = cortex.Volume(str(fs_subj_dir / 'mri' / 'T12.nii.gz'), cx_sub, 'anat', cmap='Greens', vmin=0, vmax=1)
TYPED_FRACvalue_vcx = cortex.Volume(str(GLMsingle_dir / 'TYPED_FRACvalue.nii.gz'), cx_sub, 'fmriprep', cmap='Greens', vmin=0, vmax=1)
TYPED_retinamap_vcx = cortex.Volume(str(GLMsingle_dir / 'TYPED_retinamap.nii.gz'), cx_sub, 'fmriprep', cmap='hsv', vmin=0, vmax=360)
ds = {"TYPED_FRACvalue": TYPED_FRACvalue_vcx}
ds["TYPED_retinamap"] = TYPED_retinamap_vcx
for cope, name in zip((1,2,3), ["TYPED_R2", "TYPEA_onoffR2", "TYPEC_R2"]):
    cope_fn = GLMsingle_dir / (name + '.nii.gz')
    cope_vcx = cortex.Volume(str(cope_fn), cx_sub, 'fmriprep', cmap='coolwarm')
    ds[name] = cope_vcx

cortex.webgl.show(ds)
cortex.webgl.make_static(ctx_dir / sub, ds)

Generating new ctm file...
wm
wm
inflated
inflated
Started server on port 18831


Stopping server
